<a href="https://colab.research.google.com/github/gogimandu1019/bigdata202406/blob/main/20240518_mission5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'big-data-analytics-certification-kr-2023-5th:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F54007%2F5874615%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240519%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240519T101656Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D54c975b630762aebc1e833dc744efb7590ed9e447e074ef83e5b88a475d8c224462d55f6894ade9546069894947a8593c7bcfcd3d9cf705f897c6903e45daffb938cf26a912e9cf9ef0dd74d38a435ca0db46cd342e88af7394bb53e72b4d097e804f17e0f19963d114676111c766ebdde69bd9b486602b11dd37f4d0ff0220091989d4dbdd664ad91715d4f8df5ad49e675010e8dafe2f005c0391f087f93cea165f42b9d30179d8a9df1de00d98259c5a6b5add48d4696b027eeda58cacda4d534fea14e8c0e9ae7ccce1c3c78148f357e170e51aa8677355fb045fbeb1501e8e057ff43eac965b1bc28574954684052ef126ad1246d61b12b46b73288d158'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 56891 bytes downloaded
Downloaded and uncompressed: big-data-analytics-certification-kr-2023-5th
Data source import complete.


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

train = pd.read_csv("/kaggle/input/big-data-analytics-certification-kr-2023-5th/train.csv")
test = pd.read_csv("/kaggle/input/big-data-analytics-certification-kr-2023-5th/test.csv")

#train.info()
#test.info()
#train.isnull().sum()
#test.isnull().sum()

#train.describe() #중고차가 2060년? 이건 문제가 있다
#test.describe()

cond = train['year'] >= datetime.today().year
#print(len(train[cond])) -> 1마리면 날리자

train = train[~cond]
#train.describe()

cols = ['model','transmission','fuelType']
#train[cols].nunique() -> 라벨인코딩하자 19개는 원핫 선 심하게 넘는다
#라벨인코딩
from sklearn.preprocessing import LabelEncoder

for col in cols:
  le = LabelEncoder()
  train[col] = le.fit_transform(train[col])
  test[col] = le.transform(test[col])

#train.head()
#검증용데이터 분리
from sklearn.model_selection import train_test_split

x_tr, x_val, y_tr, y_val = train_test_split(
    train.drop('price', axis = 1), #train
    train['price'], #target
    test_size = 0.2, #rate
    random_state = 1 #randomstate
)

#x_tr.head()

#학습 - randomforestregressor
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(x_tr, y_tr)
pre = model.predict(x_val)



# 평가
from sklearn.metrics import mean_squared_error
def rmse(y_test, y_pred): #RMSE
    return np.sqrt(mean_squared_error(y_test, y_pred))

# 선형회귀
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(x_tr, y_tr)
pred = model.predict(x_val)

print(rmse(y_val, pred)) #2342.4126164617383


pred = model.predict(test)
pred = pred.astype('int')
print(pred.dtype)
pred
result = pd.DataFrame({'id':test.index,'price':pred})
result.to_csv('submission.csv',index=False)
a = pd.read_csv('submission.csv')
print(a)

/kaggle/input/big-data-analytics-certification-kr-2023-5th/sample_submission.csv
/kaggle/input/big-data-analytics-certification-kr-2023-5th/test.csv
/kaggle/input/big-data-analytics-certification-kr-2023-5th/train.csv
2342.4126164617383
int64
        id  price
0        0  14832
1        1  16264
2        2  14609
3        3  15128
4        4   6186
...    ...    ...
1612  1612  11441
1613  1613  16629
1614  1614  11221
1615  1615  14981
1616  1616   5511

[1617 rows x 2 columns]
